In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, SessionNotCreatedException
from time import sleep
# from tqdm import tqdm
import pandas as pd
import time
import json

In [14]:
def get_product_content(page_link,class_name):
    driver = webdriver.Safari()
    driver.get(page_link)
    section_elements = driver.find_elements(By.CLASS_NAME, class_name)
    for div in section_elements:
        p_tags = div.find_elements(By.TAG_NAME, 'p')
        for p in p_tags:
            yield p.text
    sleep(5)
    driver.quit()

In [15]:
def get_links_btn(page_link,class_name):
    driver = webdriver.Safari()
    driver.get(page_link)
    div_elements = []
    for c in class_name:
        div_elements += driver.find_elements(By.CLASS_NAME, c)
    for div in div_elements:
        a_tag = div.find_elements(By.TAG_NAME, 'a')
        for a in a_tag:
            yield a.get_attribute('href')
        # print(a_tag.text, a_tag.get_attribute('href'))
    sleep(2)
    driver.quit()

In [16]:
def get_num_last_page(link,ul_class='pagination_page_number'):
    driver = webdriver.Safari()
    driver.get(link)

    while (True):
        try:
            # Find the button element by its class
            load_more_button = driver.find_element(By.XPATH, '//a[@aria-label="Go to next page"]')
            load_more_button.click()
            print("Clicked 'Next Page' button")
            sleep(5)
        except :
            ul = driver.find_element(By.CLASS_NAME, ul_class)
            li_elements = ul.find_elements(By.TAG_NAME, 'li')
            last_page = int(li_elements[-1].text.strip())
            break

    driver.quit()
    return last_page

In [17]:
def get_links_blogs(link,last_page):
    links = []
    for num_page in range(last_page):
        page_link = f"{link}/{num_page+1}"
        sleep(5)
        for l in get_links_btn(page_link,'blog_card_header'):
            links.append(l)
    return links

In [18]:
def check_exist_class(page_link,class_name):
    driver = webdriver.Safari()
    driver.get(page_link)
    for name in class_name:
        try:
            element = driver.find_element(By.CLASS_NAME, name)
            if element:
                yield name
        except:
            pass

    sleep(2)
    driver.quit()

Footer

In [19]:
driver = webdriver.Safari()
driver.get("https://www.innovestx.co.th")

links = []
h3_element = driver.find_elements(By.CSS_SELECTOR, 'h3.head')

for h3 in h3_element:
    try:
        h3.click()
        print('click')
        div_elements = driver.find_element(By.CLASS_NAME, 'footer_section_one')
        a_tag = div_elements.find_elements(By.TAG_NAME, 'a')
        for a in a_tag:
            links.append(a.get_attribute('href'))
        sleep(2)
    except:
        pass

driver.quit()

click
click
click
click
click


In [20]:
urls = list(set(links))
urls.sort()
urls.reverse()
visited_links = set()

data = {
        "visited_links" : list(visited_links),
        "links" : urls
    }

file_path = "InnovestX/InnovestX_config.json"

# with open(file_path, "w") as json_file:
#     json.dump(data, json_file, indent=4)
# df = pd.DataFrame({'content':[],'link':[]})
# n = 0


Scraping Data

In [27]:
with open(file_path, "r") as json_file:
    data = json.load(json_file)
    visited_links = set(data['visited_links'])
    links = data['links']

In [28]:
visited_links

{'https://www.innovestx.co.th/Invx-point-promotions',
 'https://www.innovestx.co.th/Invx-point-promotions#',
 'https://www.innovestx.co.th/about-us/board-of-director-pasu',
 'https://www.innovestx.co.th/about-us/board-of-director-prakid',
 'https://www.innovestx.co.th/about-us/board-of-director-wallaya',
 'https://www.innovestx.co.th/about-us/board-of-directors',
 'https://www.innovestx.co.th/about-us/management',
 'https://www.innovestx.co.th/about-us/rewards-innovestx',
 'https://www.innovestx.co.th/about-us/risk-committee',
 'https://www.innovestx.co.th/contact-us',
 'https://www.innovestx.co.th/innovestx-documents/announcement/-in-category/categories/announcement/fund-announcement',
 'https://www.innovestx.co.th/innovestx-documents/announcement/-in-category/categories/announcement/interest-announcement',
 'https://www.innovestx.co.th/innovestx-documents/announcement/-in-category/categories/announcement/stock-announcement',
 'https://www.innovestx.co.th/jobs',
 'https://www.innovest

In [29]:
links

['javascript:void(0);',
 'https://www.innovestx.co.th/knowledge-hub/start-your-first-investment?taxonomy=categories&propertyName=category&taxon=%2fknowledgehub%2fstart-your-first-investment%2fstartinvest',
 'https://www.innovestx.co.th/knowledge-hub/investment-by-asset?taxonomy=categories&propertyName=category&taxon=%2fknowledgehub%2finvestment-by-asset%2fstock',
 'mailto:InnovestX_Contact@scb.co.th',
 'https://www.innovestx.co.th/knowledge-hub/detail/start-investing-the-first-step/what-is-etf',
 'https://www.innovestx.co.th/knowledge-hub/detail/research-digital-asset/invx-digital-assets-daily-28042023',
 'https://www.innovestx.co.th/docs/default-source/announcement/%E0%B9%81%E0%B8%88-%E0%B8%87%E0%B8%A3%E0%B8%B0%E0%B8%87-%E0%B8%9A%E0%B8%9A%E0%B8%A3-%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%8B-%E0%B8%AD%E0%B8%82%E0%B8%B2%E0%B8%A2%E0%B8%AB%E0%B8%A5-%E0%B8%81%E0%B8%97%E0%B8%A3-%E0%B8%9E%E0%B8%A2-ptp-%E0%B9%83%E0%B8%99%E0%B8%95%E0%B8%A5%E0%B8%B2%E0%B8%94-us-24052023.pdf?sfvrsn=158ad852_3',
 'https

In [31]:
href_class = ['blog_card_header','invest_guide ','swiper-wrapper','inner-plan-inverse', 'blog_list','container','pagination_page_number']

start_time = time.time()
n = 280
df = pd.read_csv(f'InnovestX/InnovestX_{n}records.csv')

while links:
    current_url = links.pop(0)
    content = " "
    
    if current_url in visited_links or current_url[:27] != "https://www.innovestx.co.th":
        continue

    print(current_url)
    n += 1
    
    if current_url[:32] == "https://www.innovestx.co.th/docs":
        new_record = ["Document", current_url]
        df.loc[len(df.index)] = new_record
        continue

    visited_links.add(current_url)
    

    exist_class = list(check_exist_class(current_url,href_class))
    
    # print(visited_links)
    # Block links
    if "pagination_page_number" in exist_class:
        try:
            last_page = get_num_last_page(current_url)
            links += get_links_blogs(current_url,last_page)
            print(last_page)

        except :
            # print('Num page not found')
            pass
    
    
    print(exist_class)
    sleep(5)
    try:
        links += list(get_links_btn(current_url,exist_class))
    except:
        pass
    
    if 'container' in exist_class:
        content = " ".join([text.strip() for text in get_product_content(current_url,'container')])

    links = list(set(links))

    new_record = [content, current_url]
    df.loc[len(df.index)] = new_record
    print(f"exist urls: {len(links)}")
    print(f"visited urls: {len(visited_links)}\n")
    if not n%20:

        data = {
            "visited_links" : list(visited_links),
            "links" : links
        }
        with open("InnovestX/InnovestX_config.json", "w") as json_file:
            json.dump(data, json_file, indent=4)
         
        elapsed_time = time.time() - start_time
        print(f'write file using {(elapsed_time/60):.2f} minutes')
        df.to_csv(f'InnovestX/InnovestX_{n}records.csv', index=False,encoding='utf-8-sig')


https://www.innovestx.co.th/knowledge-hub/detail/research-digital-asset/invx-digital-assets-daily-28042023
['container']
exist urls: 198
visited urls: 111

https://www.innovestx.co.th/docs/default-source/announcement/%E0%B9%81%E0%B8%88-%E0%B8%87%E0%B8%A3%E0%B8%B0%E0%B8%87-%E0%B8%9A%E0%B8%9A%E0%B8%A3-%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%8B-%E0%B8%AD%E0%B8%82%E0%B8%B2%E0%B8%A2%E0%B8%AB%E0%B8%A5-%E0%B8%81%E0%B8%97%E0%B8%A3-%E0%B8%9E%E0%B8%A2-ptp-%E0%B9%83%E0%B8%99%E0%B8%95%E0%B8%A5%E0%B8%B2%E0%B8%94-us-24052023.pdf?sfvrsn=158ad852_3
https://www.innovestx.co.th/news/detail/innovestx-news-20122022
['blog_card_header', 'swiper-wrapper', 'container']
exist urls: 194
visited urls: 112

https://www.innovestx.co.th/news/detail/innovestx-Award
['blog_card_header', 'swiper-wrapper', 'container']
exist urls: 195
visited urls: 113

https://www.innovestx.co.th/news/detail/innovestx-tokenx
['blog_card_header', 'swiper-wrapper', 'container']
exist urls: 194
visited urls: 114

https://www.innovestx.co.th/n